<a href="https://colab.research.google.com/github/LaertesPecker/Clasificadores/blob/master/Practica1_Reconocimiento_Formas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRÁCTICA 1: Clasificadores Euclídeo y Bayesiano


---



## 1. Introducción



## 2. Clasificador de la distancia euclídea

### 2.1 En qué consiste el clasificador y como lo hemos implementado

El clasificador de la distancia euclídea parte de la hipótesis de que dispersion de las clases es pequeña en relación a la distancia entre ellas. Una vez aceptada la hipótesis, el representante de cada clase será el centroide de todos los datos pertenecientes a esa clase. Por tanto, el primer paso para impementar en clasificador será calcular los centroides de los datos de test. Una vez calculados los centroides, para determinar la pertenencia de un dato a una clase, calcularemos la distancia del dato a todos los centroides y el dato pertenecerá a la clase que representa el centroide más cercano a él. 

Matemáticamente, el cálculo del centroide es: $z_i = \frac{1}{card(\alpha_i)}\sum_{x \in \alpha_i} x$, donde $\alpha_i$ representa a la clase i. Como ya se ha expuesto arriba, la función discriminante será la distancia euclídea $f_i(x)=d_E(x,z_i)$. Por tanto, se clasificará un objeto en la clase cuya distancia
sea la mínima, es decir $x \in \alpha_i \leftrightarrow arg min_i{f_i(x)}$

### 2.2 Implementación del clasificador

Para implementar el clasificador se deberán definir 4 funciones: La función de **fit**, la cual calculará los centroides de nuestros datos de test. La función **predict**, la cual calculará la distancia de todos nuestros datos los cuales queremos obtener su clase a todos los centroides calculados en la función fit. La función **pred_label**, que cogerá la etiqueta del centroide mas cercano a todos los datos que queremos predecir. Finalmente, la función **num_aciertos**, que calculará el número de aciertos de nuestro clasificador.

Empezamos importando los paquetes necesarios y creando la clase y el constructor de la misma:

In [0]:
import numpy as np
from abc import abstractmethod

class Classifier:
    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifEuclid(Classifier):
    def __init__(self,labels=[]):
        self.labels=labels
        pass

A continuación, definimos la función fit:

In [0]:
def fit(self,X,y):
        self = np.array([np.mean(X[y==l],axis=0) for l in np.unique(y)])
        return self

Como se ha descrito arriba, la función fit calcula los centroides, para ello recorre todas las x con misma etiqueta y calcula su media. A continuación se procede a escribir la función predict:

In [0]:
def predict(self,X):
        distancias = np.linalg.norm(centroides[:,np.newaxis] - X,axis=2)
        return distancias

Calcula la norma entre los centroides y los valores de x, es decir, calcula la distancia euclídea y la almacena en la variable distancias. Hay que añadiendo una nueva columna o hacer un reshape de centroides para poder operar con x, haciendo el reshape para los datos del iris deberíamos cambiar la matriz de centroides con:

```
centroides.reshape(3,1,4)
```
Continuamos definiendo la función pred_label:


In [0]:
def pred_label(self,X):
        distancias = predict(self,X)
        pred = np.argmin(distancias,axis=0)
        return pred

Finalmente, calculamos el número de aciertos y el porcentaje de aciertos de nuestro clasificador:

In [0]:
def num_aciertos(self,X,y):
        pred = pred_label(self,X)
        numAciertos = (pred == y).sum()
        porcAciertos = (pred==y).mean()*100
        print(f"Numero de aciertos: {numAciertos} , Porcetaje de Aciertos: {porcAciertos}" )
        return

Definimos una función más que utilizaremos cuando queramos utilizar una parte de los datos para entrenar nuestro clasificador, por defecto entrenaremos con el 70% de los datos y testearemos con el otro 30%.

In [0]:
def clasfEucl(X,y,name,test_size=0.3):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
  centroides = np.array([np.mean(X_train[y_train==l],axis=0) for l in np.unique(y_train)])
  distancias = np.linalg.norm(centroides[:,np.newaxis] - X_test,axis=2)
  pred = np.argmin(distancias,axis=0)
  numAciertos = (pred == y_test).sum()
  porcAciertos = (pred == y_test).mean()*100
  print(f"Datos " + name + ":")
  print(f"Numero de aciertos: {numAciertos} , Porcetaje de Aciertos: {porcAciertos}" )

### 2.3 Tabla de resultados. Discusión

Utilizamos nuestro clasificador para los datos de iris, wine y cancer. Para entrenar nuestro clasificador utilizaremos todos los datos y veremos que, aún así, no obtendremos un 100% de porcentaje de aciertos:

In [0]:
X, y  = load_iris(return_X_y=True)
clasfEucl(X,y,"Iris")

Datos Iris:
Numero de aciertos: 42 , Porcetaje de Aciertos: 93.33333333333333


In [0]:
X, y  = load_wine(return_X_y=True)
clasfEucl(X,y,"Wine")

Datos Wine:
Numero de aciertos: 40 , Porcetaje de Aciertos: 74.07407407407408


In [0]:
X, y =load_breast_cancer(return_X_y=True)
clasfEucl(X,y,"Cancer")

Datos Cancer:
Numero de aciertos: 145 , Porcetaje de Aciertos: 84.7953216374269


#### 2.3.4 Discusión



## 3. Clasificador Estadístico Bayesiano


### 3.1 En qué consiste el clasificador y como lo hemos implementado


La medida de pertenencia ahora va a estar establecida por la estadística bayesiana. Cuando usabamos un clasificador euclídeo, la dispersión debía ser pequeña dentro de una clase para que funcionara bien. En este tema vamos a resolver el problema de clasificación mediante el modelado estadístico de la distribución de las muestras en casa clase. Este clasificador solo funcionará bien para clases que se distribuyan unimodalmente. Otro inconveniente de este clasificador es que el número de parámetros crece cuadráticamente. No obstante, este clasificador es muy simple matemáticamente y se adecua a muchos problemas. A menudo, es imposible encontrar una frontera que separe perfectamente dos clases. Vamos a minimizar este error con la estadística.

Para nuestro clasificador usaremos la probabilidad a posteriori pero, gracias al teorema de Bayes, conociendo las demás podemos conocer la a posteriori: 

$P(\alpha_i|x) = \frac{p(x|\alpha_i)P(\alpha_i)}{p(x)}$

Una forma de clasificar, por ejemplo, seria asignarle la que tenga mayor numerador. Por tanto, la funcion discriminante la podremos escribir como:
    
$f_j(x) = P(\alpha_j|x)$

$f_j(x) = p(\alpha_j,x) = p(x|\alpha_j)P(\alpha_j)$ (conjunta)

Ambas son compatibles porque solo difieren en una constante, p(x).

Vamos a suponer que la condicionada en cada clase sigue una gausiana normal de media μ. Entrenar nuestro clasificador sera encontrar la media y la variancia para cada clase. En una gausiana, el exponente es parecido a una distancia (si sigma vale uno o es la identidad es una distancia). Una vez calculemos la media, varianza, probabilidad a priori y la inversa de la varianza, sustituiremos los valores en la función discriminante:

$d_i(x) = -\frac{1}{2}ln|\Sigma_i| - \frac{1}{2}(x-\mu_i)^T\Sigma_i^{-1}(x-\mu_i) + ln P(\alpha_i)$


 


### 3.2 Implementación del clasificador


Nuestra función **fit** se encargará de calcular la probabilidad a priori (pues es independiente de X), la matriz de covarianzas y la media de cada clase. Quedando:

In [0]:
def fit_estadistico(X,y):
  unique, counts = np.unique(y,return_counts=True)
  self.labels = unique
  prob_priori = np.divide(counts,y.sum())
  ln_priori = np.log(prob_priori)
  self.ln_apriories = ln_priori
  self.means = np.array([np.mean(X[y==l],axis=0) for l in np.unique(y)])
  cov = np.array([np.cov(X[y==l] - self.means[l], rowvar=False) for l in np.unique(y)])
  det = np.array(np.linalg.det(cov))
  self.ln_determinants = np.log(det)
  self.inv_covs = np.linalg.inv(cov)

Nuestra función predict, calculará la resta entre las X y las medias y computará la función discriminante:

In [0]:
def predict_estadistico(X,y):
  mean_x0 = np.array(X - media[:,np.newaxis])
  j=0
  result = np.empty((3,np.shape(X)[0]))
  for row in mean_x0:
    mult = row.dot(self.inv_covs[j])
    mult = (mult*row).sum(1)
    mult = mult/-2
    result[j] = -0.5*self.ln_determinants[j] + mult + self.ln_apriories[j]
    np.vstack(result)
    j+=1

La funcion predict simplemente se quedará con los argumentos máximos:

In [0]:
def predict_label_estadistico(self,X):
  pred = np.argmax(result,axis=0)
  num_aciertos = (pred == y).sum()
  media_aciertos = (pred == y).mean()*100
  print(f"Numero de Aciertos es: {num_aciertos}, Porcentaje de aciertos: {media_aciertos}")

Quedando la clase completa como:

In [0]:
import numpy as np
from abc import abstractmethod

class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifBayesiano(Classifier):
    def __init__(self):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase"""
        self.ln_apriories = None
        self.means = None
        self.ln_determinants = None
        self.inv_covs = None
    def fit_estadistico(self,X,y):
        assert X.ndim == 2 and X.shape[0] == len(y)
        unique, counts = np.unique(y,return_counts=True)
        self.labels = unique
        prob_priori = np.divide(counts,y.sum())
        ln_priori = np.log(prob_priori)
        self.ln_apriories = ln_priori
        self.means = np.array([np.mean(X[y==l],axis=0) for l in np.unique(y)])
        cov = np.array([np.cov(X[y==l] - self.means[l], rowvar=False) for l in np.unique(y)])
        det = np.array(np.linalg.det(cov))
        self.ln_determinants = np.log(det)
        self.inv_covs = np.linalg.inv(cov)
        return self
    def predict_estadistico(self,X):
        assert self.means is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.means.shape[1]
        mean_x0 = np.array(X - self.means[:,np.newaxis])
        j=0
        result = np.empty((3,np.shape(X)[0]))
        for row in mean_x0:
          mult = row.dot(self.inv_covs[j])
          mult = (mult*row).sum(1)
          mult = mult/-2   
          result[j] = -0.5*self.ln_determinants[j] + mult + self.ln_apriories[j]
          np.vstack(result)
          j+=1
        return result
    def predict_label_estadistico(self,y,grado_pertenencia):
        pred = np.argmax(grado_pertenencia,axis=0)
        num_aciertos = (pred == y).sum()
        media_aciertos = (pred == y).mean()*100
        print(f"Numero de Aciertos es: {num_aciertos}, Porcentaje de aciertos: {media_aciertos}")
    def clasfBay(X,y,test_size=0.3):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
      clasfXBayesiano = ClassifBayesiano()
      clasfXBayesiano.fit_estadistico(X_train,y_train)
      grado = clasfXBayesiano.predict_estadistico(X_test)
      clasfXBayesiano.predict_label_estadistico(y_test,grado)


### 3.3 Tabla de resultados. Discusión y comparadlos con el de distancia euclidea

Cogiendo una muestra de test del 50% obtenemos los siguientes resultados para los tres datasets estudiados:

In [0]:
from sklearn.datasets import load_iris, load_breast_cancer, load_wine
from sklearn.model_selection import train_test_split
X, y  = load_iris(return_X_y=True)
print("Datos Iris Bayesiano:")
ClassifBayesiano.clasfBay(X,y,0.5)

Datos Iris Bayesiano:
Numero de Aciertos es: 73, Porcentaje de aciertos: 97.33333333333334


In [0]:
X, y  = load_wine(return_X_y=True)
print("Datos Wine Bayesiano:")
ClassifBayesiano.clasfBay(X,y,0.5)

Datos Wine Bayesiano:
Numero de Aciertos es: 85, Porcentaje de aciertos: 95.50561797752809


In [0]:
X, y  = load_breast_cancer(return_X_y=True)
print("Datos Cancer Bayesiano:")
ClassifBayesiano.clasfBay(X,y,0.5)

Datos Cancer Bayesiano:
Numero de Aciertos es: 259, Porcentaje de aciertos: 90.87719298245615


## 4. Regularización


### 4.1 Explica brevemente en qué consiste la regularización y justifica su necesidad.

### 4.1 Implementa el clasificador Estadístico Bayesiano Paramétrico

## Evaluación del Rendimiento

Empleando el código de soporte que se ha proporcionado en los esqueletos de las prácticas parciales, entrenay evalúa el rendimiento de los clasificadores de la distancia euclídea y del clasificador estadístico bayesiano con regularización, empleando validación cruzada k-fold, en las bases de datos de Iris, Wine y Cancer. Compara los resultados con los obtenidos en las secciones anteriores y discute las diferencias.Empleando el código de soporte que se ha proporcionado en los esqueletos de las prácticas parciales, entrena, encontrando los parámetros de regularización que obtienen el mejor rendimiento para el clasificador bayesiano paramétrico, y evalúa el rendimiento de los clasificadores de la distancia euclídea y del clasificador bayesiano paramétrico, empleando el método de exclusión en las bases de datos MNIST e Isolet.Compara los resultados con los obtenidos en las secciones anteriores y discute las diferencias.Para entrenar un clasificador y hacer una correcta evaluación de su rendimiento es necesario que el conjunto de datos que se utiliza para evaluación (conjunto de test) no se haya utilizado en ninguna etapa del entrenamiento. En caso contrario estaríamos sobre-ajustando a dicho conjunto de test. En este apartado, para simplificar el proceso se os ha propuesto entrenar y seleccionar los parámetros de regularización empleando un método wrapper que optimizase el rendimiento del conjunto de test. Razona por qué esta estimación del rendimiento está sesgada y qué habría que hacer para obtener una estimación más ajustada.

## Aplicación en un caso real de reconocimiento de texto

Emplea los clasificadores construidos en el apartado anterior para crear un programa capaz de reconocer el texto presente en una hoja de texto escaneada como la que se puede encontrar en el fichero “EJEMPLO_PRACTICA6.JPG” de la práctica 6. Puedes emplear todo el código de soporte de la Práctica 6, si así lo deseas. Describe el proceso de análisis que has seguido para mejorar tus resultados respecto a los resultados de partida. Motiva cada una de las decisiones tomadas. Describe cómo se han evaluado los resultados, si se han empleado otras imágenes tomadas por el estudiante y si es así cómo han cambiado los resultados con respecto a los resultados del fichero “EJEMPLO_PRACTICA6.JPG”. Proporciona el resultado obtenido sobre la imagen “EJEMPLO_PRACTICA6.JPG”.

## Conclusiones
Discusión final sobre los resultados obtenidos.También son bienvenidos los comentarios que quieras hacer sobre cualquier aspecto del curso.